Установка необходимых библиотек:

In [1]:
! pip install transformers
! pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 21.3 MB/s 
     |████████████████████████████████| 7.6 MB 42.4 MB/s 
     |████████████████████████████████| 163 kB 71.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Загрузка предобработанных ранее данных:

In [3]:
answers_preprocessed = []

with open(r'/content/answers_preprocessed.txt', 'r') as fp:  # текстовый файл заранее загружается в файлы
    for line in fp:
        x = line[:-1]
        answers_preprocessed.append(x)

questions_preprocessed = []

with open(r'/content/questions_preprocessed.txt', 'r') as fp:  # текстовый файл заранее загружается в файлы
    for line in fp:
        x = line[:-1]
        questions_preprocessed.append(x)

In [4]:
answers_preprocessed[0]

'возможно доставаться слишком просто решать   опасение переключаться интерес знать   никуда деваться   становиться менее предсказуемый загадочный мужчина надолго пропадать '

In [5]:
questions_preprocessed[0]

'мужчина видеться'

Загрузка токенизатора и модели:

In [6]:
from transformers import AutoTokenizer, AutoModel
import torch


#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]  #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask


#Load AutoModel from huggingface model repository
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/sbert_large_nlu_ru")
model = AutoModel.from_pretrained("sberbank-ai/sbert_large_nlu_ru").cuda()

Downloading:   0%|          | 0.00/323 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/655 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

Векторизация ответов:

In [8]:
i = 0
k = 1000
answers_embeddings = []

while k <= 10000:
    encoded_input = tokenizer(answers_preprocessed[i:k], padding=True, truncation=True, max_length=24, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**encoded_input.to('cuda'))
    answer_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
    answers_embeddings.extend(answer_embeddings)
    i += 1000
    k += 1000

In [9]:
answers_embeddings[0]

tensor([ 0.2457, -0.1983, -0.2381,  ...,  0.7298, -0.1651, -0.1893],
       device='cuda:0')

In [10]:
type(answers_embeddings[0])

torch.Tensor

In [11]:
len(answers_embeddings)

10000

Сохранение векторизованных ответов:

In [12]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [13]:
path = '/content/gdrive/My Drive/answers_embeddings_10000.pt'
torch.save(answers_embeddings, path)

Аналогично для вопросов:

In [14]:
i = 0
k = 1000
questions_embeddings = []

while k <= 10000:
    encoded_input = tokenizer(questions_preprocessed[i:k], padding=True, truncation=True, max_length=24, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**encoded_input.to('cuda'))
    question_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
    questions_embeddings.extend(question_embeddings)
    i += 1000
    k += 1000

In [15]:
questions_embeddings[0]

tensor([ 0.0953, -0.1328, -1.2083,  ..., -0.6505,  0.0944,  0.0665],
       device='cuda:0')

In [17]:
type(questions_embeddings[0])

torch.Tensor

In [18]:
len(questions_embeddings)

10000

In [19]:
path = '/content/gdrive/My Drive/questions_embeddings_10000.pt'
torch.save(questions_embeddings, path)